# Improve C# Code Performance 

- [reference](https://medium.com/stackademic/stop-writing-slow-c-span-and-memory-are-the-upgrade-your-codebase-desperately-needs-d6d61240682b). 

In [1]:
using System;

var input = "123,45.67";
var parts = input.Split(','); //Split allocates memory for a new array, Each part becomes a new string in memory
//modern dotnet use splice with span to avoid allocations
var id = int.Parse(parts[0]);
var score = double.Parse(parts[1]);

Console.WriteLine($"ID: {id}, Score: {score}");

ID: 123, Score: 45.67


- Span<T> is a stack-only type that represents a contiguous region of arbitrary memory.  
- It can be used to work with slices of arrays, strings, or unmanaged memory without additional allocations.
- Using Span<T> can help improve performance by reducing memory allocations and garbage collection overhead, especially in high-performance scenarios.


In [2]:
{

    ReadOnlySpan<char> span = input.AsSpan(); // var 키워드 사용으로 컴파일러가 타입 추론
    var prefix = span.Split(",");
    foreach(var item in prefix)
    {
        Console.WriteLine(span[item].ToString());
    }
}

123
45.67


In [ ]:
//Span Enumerator Example

using System;
using System.Threading.Tasks;

public class Program
{
    private static readonly byte[] _array = new byte[5];

    public static void Main()
    {
        new Random(42).NextBytes(_array);
        Span<byte> span = _array;

        Task.Run( () => ClearContents() );

       EnumerateSpan(span);
    }

    public static void ClearContents()
    {
        Task.Delay(20).Wait();
        lock (_array)
        {
           Array.Clear(_array, 0, _array.Length);
        }
    }

    public static void EnumerateSpan(Span<byte> span)
    {
        lock (_array)
        {
            foreach (byte element in span) //Span<T>.Enumerator is a ref struct.
            {
                Console.WriteLine(element);
                Task.Delay(10).Wait();
            }
        }

    }
}
// The example displays output like the following:
//     62
//     23
//     186
//     0
//     0
Program.Main();

62
23
186
150
174


# ReadOnlySpan<T> 

- it cannot mutate the underlying data. It is perfect for reading strings, byte arrays, network buffers, and data you did not allocate.

# Span<T> 

- it's the mutable version. You use it when you want to write into memory. For example, formatting content into a buffer.

In [3]:
{
    var value = 12345;
    Span<char> buffer = stackalloc char[32];
    bool ok = value.TryFormat(buffer, out int written);
}

In [ ]:
//wrong buffer
{
    var buffer = new char[32]; //heap allocation
}

//No heap usage. No GC involvement. No object allocation.
{
    Span<char> buffer = stackalloc char[32];
}


In [4]:
using System;

public static class StackAllocExample
{
    public static void Main()
    {
        string original = "Hello, StackAlloc!";
        Console.WriteLine($"Original: {original}");
        
        string reversed = ReverseString(original);
        Console.WriteLine($"Reversed: {reversed}");
    }

    public static string ReverseString(string input)
    {
        if (string.IsNullOrEmpty(input)) return input;

        // 1. 힙(Heap)이 아닌 스택(Stack)에 메모리를 할당합니다.
        // new char[input.Length]를 쓰면 GC가 나중에 수거해야 하지만,
        // stackalloc은 메서드가 끝나면 즉시 자동 소멸됩니다.
        Span<char> buffer = stackalloc char[input.Length];

        // 2. 데이터를 뒤집어서 스택 버퍼에 저장합니다.
        for (int i = 0; i < input.Length; i++)
        {
            buffer[i] = input[input.Length - 1 - i];
        }

        // 3. 최종 결과만 문자열(힙)로 변환하여 반환합니다.
        // 중간 연산 과정에서는 힙 할당이 전혀 발생하지 않았습니다.
        return buffer.ToString();
    }
}
StackAllocExample.Main();

Original: Hello, StackAlloc!
Reversed: !collAkcatS ,olleH


# stackalloc을 활용한 고성능 메모리 할당 예제
`stackalloc`은 힙(Heap)이 아닌 **스택(Stack)** 메모리에 배열을 할당하는 키워드입니다. 이를 `Span<T>`과 함께 사용하면 가비지 컬렉터(GC)의 관리 없이 매우 빠르게 임시 버퍼를 생성하고 사용할 수 있습니다. 주로 짧은 시간 동안 유지되는 데이터 처리에 사용됩니다.

## 예시 파일
[Microsoft 공식 stackalloc 예제 코드](https://github.com/dotnet/samples/blob/main/snippets/csharp/language-reference/operators/Stackalloc/Program.cs)

## 답변
`stackalloc`을 사용하면 `new` 키워드를 사용할 때 발생하는 힙 메모리 할당과 나중에 GC가 이를 청소하는 비용을 절약할 수 있습니다.

아래는 문자열을 뒤집는(Reverse) 기능을 구현할 때, 중간 임시 저장소를 힙 배열(`char[]`) 대신 `stackalloc`을 사용하여 최적화한 예제입니다.

### 핵심 포인트
1.  **속도 향상**: 스택 할당은 CPU 명령 몇 개로 끝나므로 힙 할당보다 훨씬 빠릅니다.
2.  **GC 부담 없음**: 메서드(`ReverseString`)가 종료되면 스택 메모리는 자동으로 정리되므로 GC가 관여하지 않습니다.
3.  **Span과의 결합**: 과거에는 `unsafe` 키워드와 포인터를 써야 했지만, 현재는 `Span<T>` 덕분에 안전하게(Safe context) 사용할 수 있습니다.
4.  **주의사항**: 스택 메모리는 크기가 작습니다(보통 1MB). 너무 큰 배열(예: 수만 개 이상의 요소)을 `stackalloc`으로 할당하면 **StackOverflowException**이 발생하여 프로그램이 종료될 수 있습니다. 따라서 작은 크기의 버퍼에만 사용해야 합니다.

### 추가 자료
- [stackalloc 식(C# 참조)](https://learn.microsoft.com/ko-kr/dotnet/csharp/language-reference/operators/stackalloc)
- [Span<T> 및 메모리 효율성 가이드](https://learn.microsoft.com/ko-kr/dotnet/standard/memory-and-spans/memory-and-spans-usage-guidelines)